# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import csv
import datetime
import time
from citipy import citipy
import random
from scipy.stats import linregress

# Google API Key
from api_keys import g_key

## Generate Cities List

In [2]:
# Setup the ranges for latitude and longitude
latitude_range = (-90,90)
longatude_range = (-180,180)

# Create variables for data
cities = []

no_cities = 2000
for cntr in range(no_cities):
    latitude = random.uniform(latitude_range[0],latitude_range[1])
    longitude = random.uniform(longatude_range[0],longatude_range[1])
    
    # If we have chosen a latitude of 0 then we don't want to use that
    # one
    if latitude != 0:
        city = citipy.nearest_city(latitude,longitude)
        city_name = city.city_name
        city_country = city.country_code
        if not any(c['name'] == city_name and c['country'] == city_country for c in cities):
            city_dict = {'name': city_name,'country': city_country}
            cities.append(city_dict)

In [3]:
print(cities)

[{'name': 'nemuro', 'country': 'jp'}, {'name': 'jamestown', 'country': 'sh'}, {'name': 'chuy', 'country': 'uy'}, {'name': 'new norfolk', 'country': 'au'}, {'name': 'yellowknife', 'country': 'ca'}, {'name': 'mantua', 'country': 'cu'}, {'name': 'hermanus', 'country': 'za'}, {'name': 'illoqqortoormiut', 'country': 'gl'}, {'name': 'nyuksenitsa', 'country': 'ru'}, {'name': 'lompoc', 'country': 'us'}, {'name': 'port elizabeth', 'country': 'za'}, {'name': 'ouegoa', 'country': 'nc'}, {'name': 'mataura', 'country': 'pf'}, {'name': 'sarangani', 'country': 'ph'}, {'name': 'bambous virieux', 'country': 'mu'}, {'name': 'sur', 'country': 'om'}, {'name': 'albany', 'country': 'au'}, {'name': 'taolanaro', 'country': 'mg'}, {'name': 'sitka', 'country': 'us'}, {'name': 'karamay', 'country': 'cn'}, {'name': 'rikitea', 'country': 'pf'}, {'name': 'meulaboh', 'country': 'id'}, {'name': 'lebu', 'country': 'cl'}, {'name': 'port-de-bouc', 'country': 'fr'}, {'name': 'benguela', 'country': 'ao'}, {'name': 'varade

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [ ]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
city_id = []
city_name = []
city_cloudiness = []
city_country = []
city_date = []
city_humidity = []
city_latitude = []
city_longitude = []
city_temperature = []
city_wind_speed = []


# Build partial query URL
query_url = f"{base_url}appid={g_key}&units={units}&q="

city_cntr = -1
no_cities_to_retrieve = 525


# Loop through cities
for index, city in enumerate(cities):
    # See if we need to sleep so we don't go over our 60 a minute max for the api
    q, mod = divmod(index+1,55)
    if mod == 0:
        print("Pausing for a minute so we don't max out our api per minutes limit!")
        time.sleep(60)
    
    
    
    # Retrieve data information
    try:
        city_json = requests.get(query_url + city['name'] + ', ' + city['country']).json()
    except:
        print(f"City not found - {city['name']}, {city['country']} - skipping")

    if city_json['cod'] == 200:
        city_cntr += 1
        print(f"Retrieved data for {city_cntr}-{city_json['name']}, {city_json['sys']['country']} ({city_json['coord']['lat']},{city_json['coord']['lon']})")
        
        city_id.append(city_cntr)
        city_name.append(city_json['name'])
        city_cloudiness.append(city_json['clouds']['all'])
        city_country.append(city_json['sys']['country'])
        city_date.append(city_json['dt'])
        city_humidity.append(city_json['main']['humidity'])
        city_latitude.append(city_json['coord']['lat'])
        city_longitude.append(city_json['coord']['lon'])
        city_temperature.append(city_json['main']['temp_max'])              
        city_wind_speed.append(city_json['wind']['speed'])
    
    if city_cntr+1 == no_cities_to_retrieve:
        break
    
          

Retrieved data for 0-Nemuro, JP (43.32,145.57)
Retrieved data for 1-Jamestown, SH (-15.94,-5.72)
Retrieved data for 2-Chui, UY (-33.7,-53.46)
Retrieved data for 3-New Norfolk, AU (-42.78,147.06)
Retrieved data for 4-Yellowknife, CA (62.46,-114.35)
Retrieved data for 5-Mantua, CU (22.29,-84.28)
Retrieved data for 6-Hermanus, ZA (-34.42,19.23)
Retrieved data for 7-Nyuksenitsa, RU (60.41,44.24)
Retrieved data for 8-Lompoc, US (34.64,-120.46)
Retrieved data for 9-Port Elizabeth, ZA (-33.92,25.57)
Retrieved data for 10-Ouégoa, NC (-20.35,164.43)
Retrieved data for 11-Sarangani, PH (5.4,125.46)
Retrieved data for 12-Bambous Virieux, MU (-20.34,57.76)
Retrieved data for 13-Sur, OM (22.57,59.53)
Retrieved data for 14-Albany, AU (-35.02,117.89)
Retrieved data for 15-Sitka, US (57.05,-135.33)
Retrieved data for 16-Rikitea, PF (-23.12,-134.97)
Retrieved data for 17-Meulaboh, ID (4.14,96.13)
Retrieved data for 18-Lebu, CL (-37.62,-73.65)
Retrieved data for 19-Port-de-Bouc, FR (43.4,4.98)
Retrieved

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
city_data = {
             "City ID": city_id,
             "City": city_name,
             "Cloudiness (%)": city_cloudiness,
             "Country": city_country,    
             "Date": city_date,
             "Humidity (%)": city_humidity,
             "Lat": city_latitude,
             "Lng": city_longitude,
             "Max Temp": city_temperature,
             "Wind Speed (mph)": city_wind_speed
            }

city_df = pd.DataFrame(city_data)
city_df.set_index('City ID', inplace = True)

# Write the cities.csv file
csv_file_name = "../output_data/cities.csv"

city_df.to_csv(csv_file_name)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression